In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
dfes = pd.read_csv('RiverRoad_Quan_RF.csv', index_col=0, parse_dates=True)
dfes.head()

qu_dem_MIN  qu_dem_MAX  qu_dem_RANGE  qu_dem_MEAN  qu_dem_STD  \
GRID_ID                                                                  
AA-10      0.691847    1.731059      1.039212     1.117221    0.202755   
AA-11      0.641888    1.487661      0.845773     1.059745    0.147762   
AA-12      0.770032    1.507396      0.737363     1.176038    0.160304   
AA-13      0.761148    1.413489      0.652341     1.131694    0.115760   
AA-14      0.696000    1.392386      0.696386     1.160470    0.116413   

         qu_dem_SUM  qu_dsm_MIN  qu_dsm_MAX  qu_dsm_RANGE  qu_dsm_MEAN  ...  \
GRID_ID                                                                 ...   
AA-10    313.939159    0.731374    6.256660      5.525286     2.557360  ...   
AA-11    295.668776    0.661373    6.679267      6.017894     2.126747  ...   
AA-12    329.290651    0.770032    5.681332      4.911300     1.700724  ...   
AA-13    316.874198    0.761148    1.462370      0.701221     1.130406  ...   
AA-14    322.610787    0.667013    1.392386      0.725372     1.158858  ...   

         qu_curpl_SUM  qu_curpr_MIN  qu_curpr_MAX  qu_curpr_RANGE  \
GRID_ID                                                             
AA-10     1688.477507   -561.943237    585.859314     1147.802551   
AA-11    -1129.404510   -841.298767    679.957519     1521.256287   
AA-12     -994.710507   -614.981445    614.381775     1229.363220   
AA-13    -2264.176594   -577.830017    633.895386     1211.725403   
AA-14     -571.203099   -574.029480    501.253113     1075.282593   

         qu_curpr_MEAN  qu_curpr_STD  qu_curpr_SUM  wetland_type  \
GRID_ID                                                            
AA-10        -6.984654    173.043352  -1941.733846    nonwetland   
AA-11         0.069758    168.483844     19.392675    nonwetland   
AA-12        -9.649778    201.707961  -2692.288038    nonwetland   
AA-13        -1.244892    180.105412   -347.324849    nonwetland   
AA-14        -1.556956    170.969213   -435.947672    nonwetland   

         FID_RiverRd_AOI     wetland  
GRID_ID                               
AA-10                  0  nonwetland  
AA-11                  0  nonwetland  
AA-12                  0  nonwetland  
AA-13                  0  nonwetland  
AA-14                  0  nonwetland  

[5 rows x 63 columns]

In [30]:
formula = 'wetland ~ qu_dem_MIN+qu_smdem_MAX+qu_smdem_SUM' #Put variables to the right of the tilde, and see the result at the bottom cell

In [31]:
train, test = train_test_split(dfes, test_size=.33, random_state=42)

In [32]:
model_log = smf.glm(formula = formula, data=train, family=sm.families.Binomial())
result = model_log.fit()
print(result.summary())
np.mean(dfes['qu_dem_MIN'])

                              Generalized Linear Model Regression Results                              
Dep. Variable:     ['wetland[nonwetland]', 'wetland[wetland]']   No. Observations:                 5769
Model:                                                     GLM   Df Residuals:                     5765
Model Family:                                         Binomial   Df Model:                            3
Link Function:                                           logit   Scale:                          1.0000
Method:                                                   IRLS   Log-Likelihood:                -2439.0
Date:                                         Tue, 13 Jul 2021   Deviance:                       4878.0
Time:                                                 13:51:35   Pearson chi2:                 8.49e+03
No. Iterations:                                              7                                         
Covariance Type:                                     nonrobust  

1.012418477089304

In [33]:
print("Coefficeients")
print(result.params)
print()
print("p-Values")
print(result.pvalues)
print()
print("Dependent variables")
print(result.model.endog_names)

Coefficeients
Intercept      -4.429136
qu_dem_MIN      6.068414
qu_smdem_MAX    6.004028
qu_smdem_SUM   -0.031401
dtype: float64

p-Values
Intercept       2.571371e-162
qu_dem_MIN      1.648280e-100
qu_smdem_MAX     3.340180e-49
qu_smdem_SUM     1.151441e-47
dtype: float64

Dependent variables
['wetland[nonwetland]', 'wetland[wetland]']


In [34]:
predictions = result.predict(test)
print(predictions[1:10])

GRID_ID
AT-27    0.159982
U-10     0.874747
BD-45    0.054808
F-44     0.095923
C-83     0.046494
CL-28    0.125192
AA-90    0.262448
G-62     0.046053
AC-55    0.996192
dtype: float64


In [35]:
predictions_nominal = [ "wetland" if x < 0.5 else "nonwetland" for x in predictions]
print(predictions_nominal[1:10])

['wetland', 'nonwetland', 'wetland', 'wetland', 'wetland', 'wetland', 'wetland', 'wetland', 'nonwetland']


In [36]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(test["wetland"], 
                       predictions_nominal))

[[ 654  402]
 [  50 1736]]


In [37]:
print(classification_report(test["wetland"], 
                            predictions_nominal, 
                            digits = 3))

              precision    recall  f1-score   support

  nonwetland      0.929     0.619     0.743      1056
     wetland      0.812     0.972     0.885      1786

    accuracy                          0.841      2842
   macro avg      0.870     0.796     0.814      2842
weighted avg      0.855     0.841     0.832      2842



In [38]:
print("Accuracy Score: {}".format(accuracy_score(y_pred=predictions_nominal,y_true=test['wetland'])))

Accuracy Score: 0.8409570724841661
